# 프로젝트명: 데이터 수집 실습 2 (공공데이터포털 API로 받기) 

#### 📌 requests는 API를 호출할 때 주로 사용하는 라이브러리입니다. 
- 서버에 HTTP 요청을 보낼때 주로 사용하며, GET, POST, PUT, DELETE를 사용할 수 있습니다.
- 데이터 API의 요구사항에 맞게 요청해야하며, 보통은 "GET", "POST"를 주로 사용됩니다.

#### 🚨 주의사항
- 공공데이터포털에선 운영계정을 신청해서 서비스키를 받아야합니다.
- 자세한 내용은 보통 API 사용문서를 참고합니다. 

![dddd](https://file.notion.so/f/f/4e4750b9-2973-47f2-9fe5-8330ce1f2c19/b501deab-8d96-4c0a-aca3-eb13441bb80a/%E1%84%89%E1%85%B3%E1%84%8F%E1%85%B3%E1%84%85%E1%85%B5%E1%86%AB%E1%84%89%E1%85%A3%E1%86%BA_2024-05-22_%E1%84%8B%E1%85%A9%E1%84%92%E1%85%AE_9.00.51.png?id=bcc828ed-b1c4-4818-b658-4604e9ba6c26&table=block&spaceId=4e4750b9-2973-47f2-9fe5-8330ce1f2c19&expirationTimestamp=1716472800000&signature=TxFSv1p8bqyg9WZNNQZUmgFX78b1_1cSuwGJ75amj9o&downloadName=%E1%84%89%E1%85%B3%E1%84%8F%E1%85%B3%E1%84%85%E1%85%B5%E1%86%AB%E1%84%89%E1%85%A3%E1%86%BA+2024-05-22+%E1%84%8B%E1%85%A9%E1%84%92%E1%85%AE+9.00.51.png)

### 1일 차이의 사이 값 가져오기

In [1]:
from datetime import datetime, timedelta

start_date = datetime.strptime('2023-01-01', '%Y-%m-%d')
end_date = datetime.strptime('2024-01-01', '%Y-%m-%d')

date_range = []
for i in range((end_date - start_date).days + 1):
    res_date = start_date + timedelta(days=i)
    res_date = res_date.strftime('%Y-%m-%d')
    date_range.append(res_date)


### 1년 차이의 한달 간격 가져오기 

In [2]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

start_date = datetime.strptime('2023-01-01', '%Y-%m-%d')
end_date = datetime.strptime('2024-01-01', '%Y-%m-%d')

date_range = []
current_date = start_date
while current_date < end_date:
    res_date = current_date.strftime('%Y-%m-%d')
    date_range.append(res_date)
    current_date += relativedelta(months=1)

print(date_range)

['2023-01-01', '2023-02-01', '2023-03-01', '2023-04-01', '2023-05-01', '2023-06-01', '2023-07-01', '2023-08-01', '2023-09-01', '2023-10-01', '2023-11-01', '2023-12-01']


### 기상청의 2023년 서울의 데이터 가져오기 (1년치 데이터 가져오기)

In [3]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

start_date = datetime.strptime('2023-01-01', '%Y-%m-%d')
end_date = datetime.strptime('2024-02-01', '%Y-%m-%d')

date_range = []
current_date = start_date
while current_date < end_date:
    res_date = current_date.strftime('%Y%m%d')
    date_range.append(res_date)
    current_date += relativedelta(months=1)

print(date_range)

['20230101', '20230201', '20230301', '20230401', '20230501', '20230601', '20230701', '20230801', '20230901', '20231001', '20231101', '20231201', '20240101']


In [4]:
start_end_date = []
for i in range(len(date_range) -1):
    start_end_date.append([date_range[i], date_range[i+1]])

In [5]:
from tqdm import tqdm
import requests
import pandas as pd
import time

url = 'http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList'
key = '''mN6190pLpfwzS1UNprd6307v+n1bR+G+vI0oJVxVuzRtb89yesxXkd5ml005Wvea+NzQlwWjSkx3QE1UVD3sqg=='''

dfs = []
for i in tqdm(start_end_date):
    print(i)

    params = {'serviceKey' : key, 
            'pageNo' : '1', 
            'numOfRows' : '999', 
            'dataType' : 'JSON', 
            'dataCd' : 'ASOS', ## 자료분류코드
            'dateCd' : 'HR', ## 날짜분류코드
            'startDt' : str(i[0]), ## 시작일
            'startHh' : '00', ## 시작시간
            'endDt' : str(i[1]), ## 종료일
            'endHh': '00', ## 종료시간
            'stnIds' : '108' ## 서울지역코드
            }

    response = requests.get(url, params=params)
    result = response.json()
    df = pd.DataFrame(result['response']['body']['items']['item'])
    dfs.append(df)

final_df = pd.concat(dfs, axis=0, ignore_index=True)

  0%|          | 0/12 [00:00<?, ?it/s]

['20230101', '20230201']


  8%|▊         | 1/12 [00:01<00:17,  1.59s/it]

['20230201', '20230301']


 17%|█▋        | 2/12 [00:34<03:22, 20.25s/it]

...

 83%|████████▎ | 10/12 [00:44<00:03,  1.84s/it]

['20231101', '20231201']


 92%|█████████▏| 11/12 [00:46<00:01,  1.77s/it]

['20231201', '20240101']


100%|██████████| 12/12 [00:47<00:00,  3.99s/it]


In [6]:
## final_df의 결측치 확인
final_df.isnull().sum()

tm            0
rnum          0
stnId         0
stnNm         0
ta            0
taQcflg       0
rn            0
rnQcflg       0
ws            0
wsQcflg       0
wd            0
wdQcflg       0
hm            0
hmQcflg       0
pv            0
td            0
pa            0
paQcflg       0
ps            0
psQcflg       0
ss            0
ssQcflg       0
icsr          0
dsnw          0
hr3Fhsc       0
dc10Tca       0
dc10LmcsCa    0
clfmAbbrCd    0
lcsCh         0
vs            0
gndSttCd      0
dmstMtphNo    0
ts            0
tsQcflg       0
m005Te        0
m01Te         0
m02Te         0
m03Te         0
dtype: int64

In [8]:
final_df['tm'] = pd.to_datetime(final_df['tm'])

In [10]:
final_df['date'] = final_df['tm'].dt.date
final_df['hour'] = final_df['tm'].dt.hour

In [14]:
final_df.groupby('date').size().reset_index(name='count').sort_values('count', ascending=False)

,date,count
212,2023-08-01,25
304,2023-11-01,25
273,2023-10-01,25
181,2023-07-01,25
90,2023-04-01,25
...,...,...
122,2023-05-03,24
121,2023-05-02,24
119,2023-04-30,24
118,2023-04-29,24


In [15]:
final_df[final_df['date'] == '2023-08-01']

,tm,rnum,stnId,stnNm,ta,taQcflg,rn,rnQcflg,ws,wsQcflg,...,gndSttCd,dmstMtphNo,ts,tsQcflg,m005Te,m01Te,m02Te,m03Te,date,hour


In [16]:
final_df['date'] = final_df['date'].astype(str)

In [17]:
final_df[final_df['date'] == '2023-08-01']

,tm,rnum,stnId,stnNm,ta,taQcflg,rn,rnQcflg,ws,wsQcflg,...,gndSttCd,dmstMtphNo,ts,tsQcflg,m005Te,m01Te,m02Te,m03Te,date,hour
5094,2023-08-01 00:00:00,745,108,서울,27.7,,,9,1.2,,...,,,26.2,,29.1,29.1,28.2,28.0,2023-08-01,0
5095,2023-08-01 00:00:00,1,108,서울,27.7,,,9,1.2,,...,,,26.2,,29.1,29.1,28.2,28.0,2023-08-01,0
5096,2023-08-01 01:00:00,2,108,서울,27.1,,,,1.4,,...,,,25.6,,28.9,28.9,28.1,27.9,2023-08-01,1
5097,2023-08-01 02:00:00,3,108,서울,26.5,,,,2.2,,...,,,25.3,,28.7,28.7,28.0,27.9,2023-08-01,2
5098,2023-08-01 03:00:00,4,108,서울,26.2,,,,1.9,,...,,,25.0,,28.5,28.6,27.9,27.9,2023-08-01,3
5099,2023-08-01 04:00:00,5,108,서울,26.0,,,,1.5,,...,,,24.8,,28.3,28.4,27.7,27.8,2023-08-01,4
5100,2023-08-01 05:00:00,6,108,서울,25.7,,,,1.4,,...,,,24.6,,28.1,28.2,27.6,27.7,2023-08-01,5
5101,2023-08-01 06:00:00,7,108,서울,25.5,,,,1.9,,...,,,24.7,,27.9,28.0,27.5,27.7,2023-08-01,6
5102,2023-08-01 07:00:00,8,108,서울,26.0,,,,1.8,,...,,,25.9,,27.8,27.9,27.4,27.6,2023-08-01,7
5103,2023-08-01 08:00:00,9,108,서울,27.4,,,,2.0,,...,,,29.9,,27.8,27.8,27.3,27.5,2023-08-01,8
